In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import holoviews as hv
import panel as pn

In [ ]:
pn.extension()
hv.extension("bokeh")

In [ ]:
def data_prep(df):
    
    df_out = pd.DataFrame()
    
    df['hfls'] = df['hfls']*(-1)
    df['hfss'] = df['hfss']*(-1)
    df['rlus'] = df['rlus']*(-1)
    df['rsus'] = df['rsus']*(-1)
    
    calc_acc = df
    
    df_out['netLW'] = df['rlus']+df['rlds']
    df_out['netSW'] = df['rsus']+df['rsds']
    df_out['hfls'] = df['hfls']
    df_out['hfss'] = df['hfss']
    
    df_out = df_out.cumsum()
    
    
    calc_acc = calc_acc.sum(axis=1)
    calc_acc = calc_acc.cumsum()
    df_out['accumulated'] = calc_acc
    
    #number_adds = pd.Series(np.arange(1, len(df_out)+1), df_out.index)
    #df_out = df_out.divide(number_adds, axis = 0)
    
    
    return df_out


In [ ]:
def line_plot(data):
    
    plots = []
    for column in data.columns:
        sub = hv.Curve((data.index,data[column]),label=column)
        plots.append(sub)

    plot = hv.Overlay(plots)
    zero = hv.HLine(0)
    zero.opts(
        color = 'black',
        line_width=1.0
    )
    
    plot = plot * zero
    return plot

In [ ]:
def get_plot(type_choice):
    
    variables = "hfls,hfss,rlus,rlds,rsus,rsds"

    ds = xr.open_dataset("https://thredds.met.no/thredds/dodsC/alertness/YOPP_supersite/ifs-ecmwf/sodankyla/sodankyla_ifs-ecmwf_2018033112.nc?time,"+variables)
    
    st = ds['time'][0].values - (ds['time'][1].values-ds['time'][0].values)
    time_index = ds['time'].values
    time_index = np.insert(time_index, 0, st, axis=0)
    
    df = ds.to_dataframe()
    df = df.reindex(time_index, fill_value=0)
    
    data = data_prep(df)
    
    plot = line_plot(data)
    
    plot.opts(title="This is a plot",
        ylabel="These are units",
        xlabel="This is time",
        width=600,
        height=400,
        legend_position='left'
        ) 
    return plot



In [ ]:
pn.Column("insert name here",get_plot("stacke"), width_policy="max")